<a href="https://colab.research.google.com/github/Jashwanth63/Custom-Model-for-classification-of-Image-Data-without-using-Convolutional-Neural-Networks/blob/main/DLL_Assingment_TYPE2_Personal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
print(torch.__version__)

1.10.0+cu111


In [403]:
!pip install torch_optimizer

In [4]:
import pickle
import torch
import torch.nn as nn
from torchvision.datasets import FashionMNIST
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import torchvision
from torchvision import transforms
from torch.utils.data import TensorDataset
import torch.nn.functional as F
import torch.optim as optim
import tqdm as tqdm
import torch_optimizer as optim

In [5]:
if torch.cuda.is_available():
  device = torch.device("cuda:0")
else:
  device = torch.device("cpu")

In [6]:
device

device(type='cuda', index=0)

In [7]:
def write_vectorised(data, data_usecase):
  if data_usecase == "TRAIN":
    with open("vectorised_train.txt", "wb") as file_pointer:
      pickle.dump(data, file_pointer)
  elif data_usecase == "TEST":
    with open("vectorised_test.txt","wb") as file_pointer:
      pickle.dump(data, file_pointer)
      
def read_vectorised(data_usecase):
  if data_usecase == "TRAIN":
    with open("vectorised_train.txt", "rb") as file_pointer:
      return pickle.load(file_pointer)
  elif data_usecase == "TEST":
    with open("vectorised_test.txt","rb") as file_pointer:
      return pickle.load(file_pointer)


In [8]:
def download_data(b_size=64):  
    """Download the Fashion-MNIST dataset and then load it into memory."""
    mnist_train = torchvision.datasets.FashionMNIST(".", train = True, transform = transforms.ToTensor(), target_transform= transforms.Lambda(lambda x:torch.zeros(
            10, dtype = torch.float64).scatter_(dim = 0, index= torch.tensor(x), value = 1)) ,download = True)
    mnist_train = torch.utils.data.DataLoader(mnist_train, batch_size = b_size, shuffle = True, num_workers = 3)

    mnist_test = torchvision.datasets.FashionMNIST(".", train = False, transform = transforms.ToTensor(), target_transform = transforms.Lambda(
        lambda x:torch.zeros(10, dtype = torch.float64).scatter_(dim=0, index = torch.tensor(x), value = 1)), download = True)
    mnist_test = torch.utils.data.DataLoader(mnist_test, batch_size=b_size, shuffle = True, num_workers = 3)
    return mnist_train, mnist_test  

In [9]:
training,  testing = download_data(b_size=64)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/raw



In [10]:
def patch_tensor_maker(image, patch_size, im_size):
  elements = []
  for l in range(0, im_size-patch_size+1, patch_size):
    for k in range(0, im_size-patch_size+1, patch_size):
      patch = []
      for i in range(patch_size):
        temp = []
        for j in range(patch_size):
          temp.append(image[l+i][k+j])
        patch.append(temp)
      elements.append(patch)
  return elements


def patch_maker(data, patch_size = 4):
  data_list_main = list()
  for i, batch in enumerate(data):
    batch_vectorised = []
    for image in batch[0]:
      im_size = image.size()[1]
      batch_vectorised.append(patch_tensor_maker(image.view(im_size, im_size), patch_size, im_size))
    main_data = torch.Tensor(batch_vectorised)
    main_labels = batch[1]
    data_list_main.append([main_data, main_labels])
    #data_list_main= TensorDataset([main_data, main_labels])
  #return torch.utils.data.DataLoader(iter(data_list_main))
  return data_list_main

In [11]:
train_data = patch_maker(training)
test_data = patch_maker(testing)

In [12]:
#Vectorize (4,4) => (16)
for i in range(len(train_data)):
    train_data[i][0] = train_data[i][0].view(-1, 49, 16) 

for i in range(len(test_data)):
    test_data[i][0] = test_data[i][0].view(-1, 49, 16) 

In [13]:
write_vectorised(train_data, data_usecase = "TRAIN")
write_vectorised(test_data, data_usecase = "TEST")


In [14]:
train_data = read_vectorised(data_usecase = "TRAIN") 
test_data = read_vectorised(data_usecase = "TEST") 

In [480]:
class net(nn.Module):
  def __init__(self, block_size):
    super(net, self).__init__()

    #STEM
    self.linear = nn.Linear(16, 512)

    #BACKBONE
    self.blocks = block_size


    #self.mlp_1 = nn.ModuleList([nn.Linear(49, 49), nn.Linear(49, 49)])
    #self.mlp_2 = nn.ModuleList([nn.Linear(256, 256), nn.Linear(256,256)])

    #self.block = nn.ModuleList([self.mlp_1, self.mlp_2]*2)

    #self.hidden1 = nn.ModuleList([nn.Linear(49,49) for i in range(self.blocks)])
    #self.hidden2 = nn.ModuleList([nn.Linear(49,49) for i in range(self.blocks)])
    #self.hidden3 = nn.ModuleList([nn.Linear(256,256) for i in range(self.blocks)])
    #self.hidden4 = nn.ModuleList([nn.Linear(256, 256) for i in range(self.blocks)])

    for i in range(self.blocks):
      self.add_module('L1_{0}'.format(i+1), nn.Linear(49,49))
      self.add_module('L2_{0}'.format(i+1), nn.Linear(49,49))
      self.add_module('L3_{0}'.format(i+1), nn.Linear(512,512))
      self.add_module('L4_{0}'.format(i+1), nn.Linear(512,512))
      self.add_module('tan1_{0}'.format(i+1), nn.Tanh())
      self.add_module('tan2_{0}'.format(i+1), nn.Tanh())



    #self.block1 = nn.ModuleList([nn.Linear(49,49), nn.Linear(49,49), nn.Linear(512,512),
    #                             nn.Linear(512,512)])
    #self.block2 = nn.ModuleList([nn.Linear(49,49), nn.Linear(49,49), nn.Linear(512,512),
    #                             nn.Linear(512,512)])
    #self.block3 = nn.ModuleList([nn.Linear(49,49), nn.Linear(49,49), nn.Linear(512, 512),
    #                             nn.Linear(512,512)])

    #self.tanactivation = nn.Tanh() #Tanh Activation does not have trainable parameters, so we reuse 
                                   #this instance when we need a tanh activation

    #self.nonlinear_block1_1 = nn.tanh()
    #self.nonlinear_block1_2 = nn.tanh()

    #self.nonlinear_block2_1 = nn.tanh()
    #self.nonlinear_block2_2 = nn.tanh()

    #CLASSIFIER
    self.output = nn.Linear(512, 10)
    #self.probs = nn.Softmax(dim=1)

    self.weight_initializer() #Initialize Weights


  def forward(self, x):
    #STEM
    x = self.linear(x)

    #BACKBONE
    #x = self.block_caller(x, self.block1)
    #x = self.block_caller(x, self.block2)
    #x = self.block_caller(x, self.block3)
    for i in range(self.blocks):
      x = self.call_block(x, block_number = i+1)

    #CLASSIFIER
    x = torch.sum(x, dim=1) / x[1].size()[0]
    x = self.output(x)
    #x = self.probs(x)
    return x

  def call_block(self, x, block_number):
    x = x.transpose(1,2)  #X=> X^T
    x = self._modules['L1_{0}'.format(block_number)](x) #matmul(X^T,W1)
    x = self._modules['tan1_{0}'.format(block_number)](x)
    #x = self.tanactivation(x) #g(X^T,W1) , g() is tanh()
    x = self._modules['L2_{0}'.format(block_number)](x) #01 = g() X W2

    x = x.transpose(1,2) #O1 => O1^T
    x = self._modules['L3_{0}'.format(block_number)](x) #matmul(01, W3)
    x = self._modules['tan2_{0}'.format(block_number)](x) #g(O1, W3), g() is tanh()
    #x = self.tanactivation(x) 
    x = self._modules['L4_{0}'.format(block_number)](x) #O2 = g() X W4
  
    return x

  def weight_initializer(self):
    for mod in self.modules():
      if isinstance(mod, nn.Linear):
        nn.init.kaiming_uniform_(mod.weight, mode = 'fan_in')
        
       # if mod.bias is not None:
       #   nn.init.constant_(mod.bias, 0)




'''
  def block_caller(self, x, block):
    #BACKBONE - EACH BLOCK HERE HAS 2 MLPS
    for i, layers in enumerate(block):
      #h1 = self.hidden1[i]
      #h2 = self.hidden2[i]
      if i==0:
        x = x.transpose(1,2)  #X=> X^T
        x = self.tanactivation(block[i](x)) #g(X^T,W1) , g() is relu()
      elif i==1:
        x = block[i](x) #g() X W2
      
      #h3 = self.hidden3[i]
      #h4 = self.hidden4[i]
      elif i==2:
        x = x.transpose(1,2) #O1 => O1^T
        x = self.tanactivation(block[i](x))  #g(O1, W3), g() is relu()
      elif i==3:
        x = block[i](x) #g() X W4
    return x
'''
  

'\n  def block_caller(self, x, block):\n    #BACKBONE - EACH BLOCK HERE HAS 2 MLPS\n    for i, layers in enumerate(block):\n      #h1 = self.hidden1[i]\n      #h2 = self.hidden2[i]\n      if i==0:\n        x = x.transpose(1,2)  #X=> X^T\n        x = self.tanactivation(block[i](x)) #g(X^T,W1) , g() is relu()\n      elif i==1:\n        x = block[i](x) #g() X W2\n      \n      #h3 = self.hidden3[i]\n      #h4 = self.hidden4[i]\n      elif i==2:\n        x = x.transpose(1,2) #O1 => O1^T\n        x = self.tanactivation(block[i](x))  #g(O1, W3), g() is relu()\n      elif i==3:\n        x = block[i](x) #g() X W4\n    return x\n'

In [481]:
NET = net(block_size=2).to(device)

In [482]:
for name in NET.modules():
  print(name, )

net(
  (linear): Linear(in_features=16, out_features=512, bias=True)
  (L1_1): Linear(in_features=49, out_features=49, bias=True)
  (L2_1): Linear(in_features=49, out_features=49, bias=True)
  (L3_1): Linear(in_features=512, out_features=512, bias=True)
  (L4_1): Linear(in_features=512, out_features=512, bias=True)
  (tan1_1): Tanh()
  (tan2_1): Tanh()
  (L1_2): Linear(in_features=49, out_features=49, bias=True)
  (L2_2): Linear(in_features=49, out_features=49, bias=True)
  (L3_2): Linear(in_features=512, out_features=512, bias=True)
  (L4_2): Linear(in_features=512, out_features=512, bias=True)
  (tan1_2): Tanh()
  (tan2_2): Tanh()
  (output): Linear(in_features=512, out_features=10, bias=True)
)
Linear(in_features=16, out_features=512, bias=True)
Linear(in_features=49, out_features=49, bias=True)
Linear(in_features=49, out_features=49, bias=True)
Linear(in_features=512, out_features=512, bias=True)
Linear(in_features=512, out_features=512, bias=True)
Tanh()
Tanh()
Linear(in_features=

In [483]:
for name, param in NET.named_parameters():
  print(name," - ", param.size())

linear.weight  -  torch.Size([512, 16])
linear.bias  -  torch.Size([512])
L1_1.weight  -  torch.Size([49, 49])
L1_1.bias  -  torch.Size([49])
L2_1.weight  -  torch.Size([49, 49])
L2_1.bias  -  torch.Size([49])
L3_1.weight  -  torch.Size([512, 512])
L3_1.bias  -  torch.Size([512])
L4_1.weight  -  torch.Size([512, 512])
L4_1.bias  -  torch.Size([512])
L1_2.weight  -  torch.Size([49, 49])
L1_2.bias  -  torch.Size([49])
L2_2.weight  -  torch.Size([49, 49])
L2_2.bias  -  torch.Size([49])
L3_2.weight  -  torch.Size([512, 512])
L3_2.bias  -  torch.Size([512])
L4_2.weight  -  torch.Size([512, 512])
L4_2.bias  -  torch.Size([512])
output.weight  -  torch.Size([10, 512])
output.bias  -  torch.Size([10])


In [484]:
optimizer = optim.SWATS(NET.parameters(), amsgrad = True, nesterov = True, weight_decay = 0.001) #SWATS optimizer, Uses Both SGD and ADAM.  
loss_function = nn.CrossEntropyLoss()

In [485]:
def accuracy_calculator(hyp, y):
  a = torch.argmax(hyp, dim=1)
  b = torch.argmax(y, dim=1)
  return torch.sum(a==b)

In [478]:
from torch.nn.modules.loss import KLDivLoss
def train(out):
  EPOCHS = 50
  accuracy = 0
  for i in range(EPOCHS):
      out.train()
      accuracy = 0
      acc = 0
      for j in tqdm.tqdm(range(len(train_data))): 
          X, y = train_data[j]
          X = X.to(device)
          y = y.to(device)
          out.zero_grad()
          hypothesis = out(X)
          hypothesis.to(device)
          accuracy += accuracy_calculator(hypothesis, y)
          loss = loss_function(hypothesis, y)
          loss.backward()
          optimizer.step()
      out.eval()
      with torch.no_grad():
        for k in range(len(test_data)):
          X_test, y_test = test_data[k]
          X_test = X_test.to(device)
          y_test = y_test.to(device)
          test_hypothesis = out(X_test)
          test_hypothesis.to(device)
          acc += accuracy_calculator(test_hypothesis, y_test)
          test_loss =  loss_function(test_hypothesis, y_test)
        print("EPOCHS : ", i+1, "Training LOSS : ",loss, "Test Loss: ", test_loss)
        print("Training Accuracy is: ",accuracy/60000, "Testing Accuracy is: ", acc/10000)
  

def test(out):
  accuracy = 0
  with torch.no_grad():
    for i in tqdm.tqdm(range(len(test_data))):
          X, y = test_data[i]
          X = X.to(device)
          y = y.to(device)
          hypothesis = out(X)
          hypothesis.to(device)
          accuracy += accuracy_calculator(hypothesis, y)
    print(accuracy/10000)

In [468]:
train(NET)

100%|██████████| 938/938 [00:13<00:00, 67.80it/s]


EPOCHS :  1 Training LOSS :  tensor(0.2973, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward1>) Test Loss:  tensor(0.4013, device='cuda:0', dtype=torch.float64)
Training Accuracy is:  tensor(0.8009, device='cuda:0') Testing Accuracy is:  tensor(0.8363, device='cuda:0')


100%|██████████| 938/938 [00:13<00:00, 68.53it/s]


EPOCHS :  2 Training LOSS :  tensor(0.2080, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward1>) Test Loss:  tensor(0.3162, device='cuda:0', dtype=torch.float64)
Training Accuracy is:  tensor(0.8509, device='cuda:0') Testing Accuracy is:  tensor(0.8466, device='cuda:0')


100%|██████████| 938/938 [00:13<00:00, 68.65it/s]


...

EPOCHS :  48 Training LOSS :  tensor(0.1199, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward1>) Test Loss:  tensor(0.2282, device='cuda:0', dtype=torch.float64)
Training Accuracy is:  tensor(0.9095, device='cuda:0') Testing Accuracy is:  tensor(0.8817, device='cuda:0')


100%|██████████| 938/938 [00:06<00:00, 149.38it/s]


EPOCHS :  49 Training LOSS :  tensor(0.1192, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward1>) Test Loss:  tensor(0.2289, device='cuda:0', dtype=torch.float64)
Training Accuracy is:  tensor(0.9096, device='cuda:0') Testing Accuracy is:  tensor(0.8816, device='cuda:0')


100%|██████████| 938/938 [00:06<00:00, 149.63it/s]


EPOCHS :  50 Training LOSS :  tensor(0.1185, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward1>) Test Loss:  tensor(0.2295, device='cuda:0', dtype=torch.float64)
Training Accuracy is:  tensor(0.9097, device='cuda:0') Testing Accuracy is:  tensor(0.8817, device='cuda:0')


In [469]:
test(NET)

100%|██████████| 157/157 [00:00<00:00, 437.37it/s]

tensor(0.8817, device='cuda:0')


In [470]:
#NOTE:
#1. EPOCHS =100, lr =0.1, BLOCKS =2, accuracy: 73 (Train), 72(Test), Activation: Relu
#2. EPOCHS =100, lr =0.1, BLOCKS =1, accuracy: 74.93 (Train), 73.18(Test), Activation: Relu
#2. EPOCHS =100, lr =0.1, BLOCKS =2, accuracy: 84.82 (Train), 82.34(Test), Activation: TanH
#2. EPOCHS =100, lr =0.13, BLOCKS =3, accuracy: 81.56 (Train), 80.02(Test), Activation: TanH

#IDEAL MODELS:
#1. EPOCHS:100, lr = 0.2, BLOCKS = 2, Accuracy: 83.39 (Train), 81.62 (Test), Activation: TanH, Train Loss: 1.5554 LOG: (01:09 AM, 08-04-2022)
#                                     Top Record: 85.01 (Train), 82.05 (Test), Loss: 1.55490 LOG: (03:07 AM, 08-04-2022)  #Automated Blocks and Used nn.Tanh()

#2. EPOCHS: 100, lr =0.2, BLOCKS = 2, Accuracy: 91.61 (Train), 86.97 (Test), Activation: TanH, Train Loss: 1.5031 / 1.4612 (Lowest) LOG: (03:39 AM, 08-04-2022)
#                                   #Used Kaiming Uniform Initialization 


#3. EPOCHS: 50, AdamW optimizer, Blocks = 2, Accuracy: 92.82 (Train), 86.92 (Test) LOG: (06:28 AM, 08-04-2022)
#4. EPOCHS: 50, SWATS optimizer, Blocks = 2, Accuracy: 98.66 (Train), 87.93 (Test), LOG: (06:54 AM, 08-04-2022)
#5*[CURRENT RECORD]. EPOCHS: 50, SWATS optimizer(L2 regularized 0.001, amsgrad = True, Nesterov = True),
#                               Blocks = 2, Accuracy: 90.97 (Train), 88.17 (Test), Training Loss: 0.1185, Test Loss: 0.2295  LOG: (07:45 AM, 08-04-2022)

In [471]:
#NOTE TO SELF:
#1. Linear Layer uses Uniform Weight Initialization, Try out Kaiming_he and Xavier Initialization
#2. I AM AN IDIOT: APPARENTLY CROSSENTROPYLOSS CALCULATES SOFTMAX OUTPUT THEN CALCULATES LOSS, I WAS DOING SOFTMAX MANUALLY. 
#                  THE MODEL CALCULATED SOFTMAX TWICE BEFORE THE OUTPUT, HENCE RESULTING IN HIGH LOSS AND LOW ACCURACY. LOG: (05:52 AM, 08-04-2022)